In [11]:
import numpy as np
import pandas as pd
import os
import re
from io import StringIO
from tqdm import tqdm

In [12]:
folder_name = 'data_summaries'
try:
    os.mkdir(folder_name)
    print("Directory" , folder_name ,  "created ") 
except FileExistsError:
    print("Directory" , folder_name ,  "already exists")

for folder in os.listdir('data'):
    with open(os.path.join(folder_name, 'summary_{}.txt'.format(str(folder))), "w") as outfile:
        for textfile in os.listdir(os.path.join('data', folder)):
            contents = open( os.path.join('data', folder, textfile) ).read()
            outfile.write(contents)

Directory data_summaries already exists


In [13]:
data_summaries = pd.read_excel('data.xlsx')
file_list = os.listdir('data_summaries')
file_list

['summary_sub10.txt',
 'summary_sub12.txt',
 'summary_sub13.txt',
 'summary_Sub15.txt',
 'summary_Sub2.txt',
 'summary_Sub31.txt',
 'summary_Sub32.txt',
 'summary_sub5.txt',
 'summary_Sub6.txt',
 'summary_sub7.txt',
 'summary_sub8.txt']

In [14]:
def listToString(s):  
    str1 = '' 
    for ele in s:  
        str1 += ele
    return str1

In [22]:
data_rows = []
columns = ['COLUMN_{}'.format(x) for x in range(14)] + ['MEASUREMENT_NO', 'TYPE_ID']
df = pd.DataFrame(columns=columns)
with open (os.path.join('data_summaries', file_list[0]), 'r') as text_file:
    string = text_file.readlines()
    for line in tqdm(string):
        line_string = listToString(line)
        if len([s for s in line_string if '\t' in s]) >= 13:
            data_rows.append(line_string)

columns = ['C_{}'.format(x) for x in range(12)] + ['TYPE_ID', 'MEASUREMENT_NO']
df = pd.read_csv(StringIO(listToString(data_rows)), sep='\t', header=None, usecols=range(14))
df.columns = columns

100%|██████████| 204712/204712 [00:02<00:00, 85777.77it/s]


In [23]:
df

,C_0,C_1,C_2,C_3,C_4,C_5,C_6,C_7,C_8,C_9,C_10,C_11,TYPE_ID,MEASUREMENT_NO
0,2370.0,2345.0,2751.0,1604.0,2711.0,2726.0,0.7271,0.5739,0.9061,0.9301,0.7411,0.9053,1.0,0.0
1,2356.0,2332.0,2740.0,1598.0,2729.0,2736.0,0.7223,0.5694,0.9021,0.9263,0.7504,0.9089,1.0,0.0
2,2373.0,2336.0,2738.0,1593.0,2722.0,2734.0,0.7282,0.5708,0.9013,0.9231,0.7468,0.9082,1.0,0.0
3,2357.0,2331.0,2741.0,1600.0,2728.0,2739.0,0.7227,0.5690,0.9024,0.9275,0.7499,0.9100,1.0,0.0
4,2368.0,2330.0,2742.0,1599.0,2729.0,2740.0,0.7265,0.5687,0.9028,0.9269,0.7504,0.9104,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204195,1790.0,1222.0,2576.0,1843.0,2910.0,2788.0,0.5278,0.1804,0.8415,1.0000,0.8439,0.9278,9.0,9.0
204196,1798.0,1227.0,2579.0,1847.0,2900.0,2783.0,0.5306,0.1822,0.8426,1.0000,0.8388,0.9260,9.0,9.0
204197,1809.0,1238.0,2588.0,1848.0,2874.0,2755.0,0.5344,0.1861,0.8459,1.0000,0.8253,0.9158,9.0,9.0
204198,1808.0,1237.0,2587.0,1832.0,2872.0,2755.0,0.5340,0.1857,0.8455,1.0000,0.8243,0.9158,9.0,9.0


In [20]:
df[df['MEASUREMENT_NO']>0]

,C_0,C_1,C_2,C_3,C_4,C_5,C_6,C_7,C_8,C_9,C_10,C_11,TYPE_ID,MEASUREMENT_NO
0,2729.0,2652.0,2708.0,1731.0,2580.0,2266.0,0.8505,0.6815,0.8902,1.0,0.6730.8533,0.685,0.8936,1.0
